In [1]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.PascalVOC import dataset_voc, prepare_dataloaders_pascal_voc
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent
from src.training.Trainer import Trainer

In [2]:
def seed_everything(seed_value=4995):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

In [3]:
root_dir = 'Data/PascalVOC/VOCdevkit/VOC2012'

In [4]:
config = {
    'lr': 0.005,
    'batch_size': 64,
    'weight_decay': 0.01
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def load_model(num_classes = 20, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        model.fc.reset_parameters()
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict

In [6]:
resnet, loaded_state_dict = load_model()

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

In [7]:
dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'])

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [8]:
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10
    )

In [9]:
trainer.train()

Current Epoch: 0
Train Loop:
Batch: 0 of 90. Loss: 0.7339275479316711. Mean so far: 0.7339275479316711. Mean of 100: 0.7339275479316711
Batch: 10 of 90. Loss: 0.3200211524963379. Mean so far: 0.3522043526172638. Mean of 100: 0.3522043526172638
Batch: 20 of 90. Loss: 0.1715962439775467. Mean so far: 0.29454260496866136. Mean of 100: 0.29454260496866136
Batch: 30 of 90. Loss: 0.13210362195968628. Mean so far: 0.25066549883734796. Mean of 100: 0.25066549883734796
Batch: 40 of 90. Loss: 0.13094887137413025. Mean so far: 0.2217899137517301. Mean of 100: 0.2217899137517301
Batch: 50 of 90. Loss: 0.12008646875619888. Mean so far: 0.20227519335115657. Mean of 100: 0.20227519335115657
Batch: 60 of 90. Loss: 0.1315528005361557. Mean so far: 0.1889433438172106. Mean of 100: 0.1889433438172106
Batch: 70 of 90. Loss: 0.1205672025680542. Mean so far: 0.18014137320955034. Mean of 100: 0.18014137320955034
Batch: 80 of 90. Loss: 0.11310606449842453. Mean so far: 0.17242838552704565. Mean of 100: 0.1724

In [14]:
resnet, loaded_state_dict = load_model(model_path = 'models/pascal_voc_resnet.pt')

for name, param in resnet.named_parameters():
    param.requires_grad = True

Loaded models/pascal_voc_resnet.pt


In [15]:
dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'])

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = 0.0001, weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [16]:
trainer = Trainer(
        model = resnet,
        loss_fn = loss_fn,
        classes = classes,
        num_epochs = 7,
        model_name = f'pascal_voc_resnet_fine_tuned',
        training_params = training_params,
        multi_label = True,
        print_frequency = 10
    )

In [17]:
trainer.train()

Current Epoch: 0
Train Loop:
Batch: 0 of 90. Loss: 0.1093747615814209. Mean so far: 0.1093747615814209. Mean of 100: 0.1093747615814209
Batch: 10 of 90. Loss: 0.10952074825763702. Mean so far: 0.10686147822575136. Mean of 100: 0.10686147822575136
Batch: 20 of 90. Loss: 0.10893751680850983. Mean so far: 0.1063223686956224. Mean of 100: 0.1063223686956224
Batch: 30 of 90. Loss: 0.10444706678390503. Mean so far: 0.10402600082658953. Mean of 100: 0.10402600082658953
Batch: 40 of 90. Loss: 0.0973462387919426. Mean so far: 0.10259103302548571. Mean of 100: 0.10259103302548571
Batch: 50 of 90. Loss: 0.08906654268503189. Mean so far: 0.10177080917592142. Mean of 100: 0.10177080917592142
Batch: 60 of 90. Loss: 0.09077568352222443. Mean so far: 0.10163964355578188. Mean of 100: 0.10163964355578188
Batch: 70 of 90. Loss: 0.08782576769590378. Mean so far: 0.10063088024166268. Mean of 100: 0.10063088024166268
Batch: 80 of 90. Loss: 0.09419504553079605. Mean so far: 0.10020388543237875. Mean of 100: